In [1]:
!pip install delta-spark==1.2.0

In [2]:
from pyspark.sql import SparkSession

In [3]:
import pyspark
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("DeltaAppCatalog") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("hive.metastore.client.factory.class","com.amazonaws.glue.catalog.metastore.AWSGlueDataCatalogHiveClientFactory") \
    .config("hive.metastore.glue.catalogid","000000000000") \
    .config("hive.metastore.uris", "thrift://localstack:4566") \
    .config("hive.metastore.warehouse.dir","s3a://mytestbucket/warehouse")

spark = configure_spark_with_delta_pip(builder).master("spark://spark:7077").getOrCreate()

spark._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "http://localstack:4566")
spark._jsc.hadoopConfiguration().set("fs.s3a.access.key", "foo")
spark._jsc.hadoopConfiguration().set("fs.s3a.secret.key", "foo")
spark._jsc.hadoopConfiguration().set("fs.s3a.impl","org.apache.hadoop.fs.s3a.S3AFileSystem")
spark._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
spark._jsc.hadoopConfiguration().set("fs.s3a.aws.credentials.provider","org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
spark._jsc.hadoopConfiguration().set("fs.s3a.aws.credentials.provider","org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")    

In [4]:
data = spark.range(0, 5)
data.write.format("delta").save("s3a://mytestbucket/table-cat")

AnalysisException: s3a://mytestbucket/table-cat already exists.

In [ ]:
df = spark.read.format("delta").load("s3a://mytestbucket/table-cat")
df.show()

In [5]:
spark.sql('show databases').show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [6]:
spark.sql('create database source').show()

++
||
++
++



In [7]:
spark.sql('create database processed').show()

++
||
++
++



In [8]:
spark.sql("create table source.tablecat using delta location 's3a://mytestbucket/table-cat'").show()

++
||
++
++



In [9]:
spark.sql("use source")
spark.sql("show tables").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|   source| tablecat|      false|
+---------+---------+-----------+



In [10]:
spark.sql("select * from source.tablecat").show()

+---+
| id|
+---+
|  2|
|  3|
|  4|
|  0|
|  1|
+---+

